# Hybrid network for classification

In [2]:
pip install lazypredict

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Imports
import tensorflow as tf
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.supervised import LazyClassifier
import numpy as np
import lazypredict

SEED = 64
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)

#from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score

/Users/manwest/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('../../Datasets/heloc.csv')
#df['CLASSI_FIN'] = df.CLASSI_FIN.replace(to_replace=['arbovirus', 'inconclusive'], value=[1, 0])
df

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,Bad,55,144,4,84,20,3,0,83,2,...,43,0,0,0,33,-8,8,1,1,69
1,Bad,61,58,15,41,2,4,4,100,-7,...,67,0,0,0,0,-8,0,-8,-8,0
2,Bad,67,66,5,24,9,0,0,100,-7,...,44,0,4,4,53,66,4,2,1,86
3,Bad,66,169,1,73,28,1,1,93,76,...,57,0,5,4,72,83,6,4,3,91
4,Bad,81,333,27,132,12,0,0,100,-7,...,25,0,1,1,51,89,3,1,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9866,Good,73,131,5,57,21,0,0,95,80,...,19,7,0,0,26,-8,5,2,0,100
9867,Bad,65,147,39,68,11,0,0,92,28,...,42,1,1,1,86,53,2,2,1,80
9868,Bad,74,129,6,64,18,1,1,100,-7,...,33,3,4,4,6,-8,5,-8,0,56
9869,Bad,72,234,12,113,42,2,2,96,35,...,20,6,0,0,19,-8,4,1,0,38


In [3]:
df

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,Bad,55,144,4,84,20,3,0,83,2,...,43,0,0,0,33,-8,8,1,1,69
1,Bad,61,58,15,41,2,4,4,100,-7,...,67,0,0,0,0,-8,0,-8,-8,0
2,Bad,67,66,5,24,9,0,0,100,-7,...,44,0,4,4,53,66,4,2,1,86
3,Bad,66,169,1,73,28,1,1,93,76,...,57,0,5,4,72,83,6,4,3,91
4,Bad,81,333,27,132,12,0,0,100,-7,...,25,0,1,1,51,89,3,1,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9866,Good,73,131,5,57,21,0,0,95,80,...,19,7,0,0,26,-8,5,2,0,100
9867,Bad,65,147,39,68,11,0,0,92,28,...,42,1,1,1,86,53,2,2,1,80
9868,Bad,74,129,6,64,18,1,1,100,-7,...,33,3,4,4,6,-8,5,-8,0,56
9869,Bad,72,234,12,113,42,2,2,96,35,...,20,6,0,0,19,-8,4,1,0,38


In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['RiskPerformance'] = encoder.fit_transform(df['RiskPerformance'])

In [5]:

categorical_columns = df.select_dtypes(include=['object', 'category']).columns
print("Categorical columns:", categorical_columns)

Categorical columns: Index([], dtype='object')


In [6]:

# Step 1: Select the first column
first_column = df.iloc[:, 0]
# Step 2: Drop the first column from the dataframe
df = df.iloc[:, 1:]
# Step 3: Append the first column to the end of the dataframe
df[first_column.name] = first_column

In [7]:
df

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance,RiskPerformance
0,55,144,4,84,20,3,0,83,2,3,...,0,0,0,33,-8,8,1,1,69,0
1,61,58,15,41,2,4,4,100,-7,0,...,0,0,0,0,-8,0,-8,-8,0,0
2,67,66,5,24,9,0,0,100,-7,7,...,0,4,4,53,66,4,2,1,86,0
3,66,169,1,73,28,1,1,93,76,6,...,0,5,4,72,83,6,4,3,91,0
4,81,333,27,132,12,0,0,100,-7,7,...,0,1,1,51,89,3,1,0,80,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9866,73,131,5,57,21,0,0,95,80,6,...,7,0,0,26,-8,5,2,0,100,1
9867,65,147,39,68,11,0,0,92,28,6,...,1,1,1,86,53,2,2,1,80,0
9868,74,129,6,64,18,1,1,100,-7,6,...,3,4,4,6,-8,5,-8,0,56,0
9869,72,234,12,113,42,2,2,96,35,6,...,6,0,0,19,-8,4,1,0,38,0


In [11]:
X = df.iloc[:,:-1].values
y = df['RiskPerformance'].values
y

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=SEED, shuffle=True)
X_train

array([[ 72, 110,   1, ...,   2,   0,  42],
       [ 57,  57,   4, ...,   3,   1,  91],
       [ 66, 273,   1, ...,  -8,   5,  58],
       ...,
       [ 88, 199,   4, ...,   1,   0, 100],
       [ 54, 235,   7, ...,   3,   3,  67],
       [ 76,  85,   5, ...,   2,   0,  80]])

In [13]:
y_test

array([1, 1, 0, ..., 1, 1, 1])

In [14]:
#import lazypredict
lazypredict.supervised.CLASSIFIERS

[('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('CalibratedClassifierCV', sklearn.calibration.CalibratedClassifierCV),
 ('CategoricalNB', sklearn.naive_bayes.CategoricalNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('ExtraTreeClassifier', sklearn.tree._classes.ExtraTreeClassifier),
 ('ExtraTreesClassifier', sklearn.ensemble._forest.ExtraTreesClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('KNeighborsClassifier',
  sklearn.neighbors._classification.KNeighborsClassifier),
 ('LabelPropagation',
  sklearn.semi_supervised._label_propagation.LabelPropagation),
 ('LabelSpreading', sklearn.semi_supervised._label_propagation.LabelSpreading),
 ('LinearDiscriminantAnalysis',
  sklearn.discriminant_analysis.LinearDiscriminantAnalysis),
 ('Linear

In [9]:
#lazypredict.supervised.REGRESSORS = lazypredict.supervised.REGRESSORS[33:36]
#lazypredict.supervised.REGRESSORS

[('RandomForestRegressor', sklearn.ensemble._forest.RandomForestRegressor),
 ('Ridge', sklearn.linear_model._ridge.Ridge),
 ('RidgeCV', sklearn.linear_model._ridge.RidgeCV)]

In [15]:
#regressor=['RandomForestRegressor', 'XGBRegressor', 'GradientBoostingRegressor', 'HistGradientBoostingRegressor']
reg = LazyClassifier(verbose=2, ignore_warnings=True, custom_metric=None)

In [16]:
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

  0%|          | 0/29 [00:00<?, ?it/s]

  3%|▎         | 1/29 [00:00<00:05,  4.87it/s]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.7326580724370779, 'Balanced Accuracy': 0.731386165577342, 'ROC AUC': 0.7313861655773422, 'F1 Score': 0.732577256683507, 'Time taken': 0.20519208908081055}


 14%|█▍        | 4/29 [00:00<00:03,  7.93it/s]

{'Model': 'BaggingClassifier', 'Accuracy': 0.7139349294045426, 'Balanced Accuracy': 0.7094669117647059, 'ROC AUC': 0.7094669117647059, 'F1 Score': 0.7120453448400237, 'Time taken': 0.2722761631011963}
{'Model': 'BernoulliNB', 'Accuracy': 0.7090239410681399, 'Balanced Accuracy': 0.7079827069716775, 'ROC AUC': 0.7079827069716775, 'F1 Score': 0.7090347259602482, 'Time taken': 0.009028196334838867}
{'Model': 'CalibratedClassifierCV', 'Accuracy': 0.7274401473296501, 'Balanced Accuracy': 0.7257557189542483, 'ROC AUC': 0.7257557189542483, 'F1 Score': 0.7272227149484084, 'Time taken': 0.10124397277832031}
{'Model': 'DecisionTreeClassifier', 'Accuracy': 0.6470227133210559, 'Balanced Accuracy': 0.6457890795206971, 'ROC AUC': 0.6457890795206972, 'F1 Score': 0.6470486102898604, 'Time taken': 0.06629586219787598}
{'Model': 'DummyClassifier', 'Accuracy': 0.5303867403314917, 'Balanced Accuracy': 0.5, 'ROC AUC': 0.5, 'F1 Score': 0.3676326864391567, 'Time taken': 0.011675119400024414}


...

{'Model': 'RidgeClassifierCV', 'Accuracy': 0.7265193370165746, 'Balanced Accuracy': 0.7251123366013071, 'ROC AUC': 0.7251123366013071, 'F1 Score': 0.7264003075073688, 'Time taken': 0.3843109607696533}
{'Model': 'SGDClassifier', 'Accuracy': 0.7071823204419889, 'Balanced Accuracy': 0.7099162581699346, 'ROC AUC': 0.7099162581699348, 'F1 Score': 0.7071485545656562, 'Time taken': 0.0438690185546875}


 90%|████████▉ | 26/29 [00:07<00:00,  3.01it/s]

{'Model': 'SVC', 'Accuracy': 0.7298956414978515, 'Balanced Accuracy': 0.7275837418300654, 'ROC AUC': 0.7275837418300654, 'F1 Score': 0.7294150114372125, 'Time taken': 1.0440120697021484}
{'Model': 'XGBClassifier', 'Accuracy': 0.7176181706568447, 'Balanced Accuracy': 0.7154105392156863, 'ROC AUC': 0.7154105392156863, 'F1 Score': 0.7171865777274471, 'Time taken': 0.0705709457397461}
[LightGBM] [Info] Number of positive: 3205, number of negative: 3408
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1465
[LightGBM] [Info] Number of data points in the train set: 6613, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.484651 -> initscore=-0.061414
[LightGBM] [Info] Start training from score -0.061414


100%|██████████| 29/29 [00:07<00:00,  3.79it/s]

{'Model': 'LGBMClassifier', 'Accuracy': 0.7271332105586249, 'Balanced Accuracy': 0.7248297930283225, 'ROC AUC': 0.7248297930283225, 'F1 Score': 0.7266564113798235, 'Time taken': 0.550710916519165}


In [17]:
models


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.74,0.73,0.73,0.73,0.42
AdaBoostClassifier,0.73,0.73,0.73,0.73,0.21
RandomForestClassifier,0.73,0.73,0.73,0.73,0.69
SVC,0.73,0.73,0.73,0.73,1.04
CalibratedClassifierCV,0.73,0.73,0.73,0.73,0.10
RidgeClassifier,0.73,0.73,0.73,0.73,0.01
LinearDiscriminantAnalysis,0.73,0.73,0.73,0.73,0.15
RidgeClassifierCV,0.73,0.73,0.73,0.73,0.38
LGBMClassifier,0.73,0.72,0.72,0.73,0.55
